# Write cache for future hydro

In [7]:
from loca import print_date
print_date()

Last executed: 2020-05-23 08:01:52.473235 by jvano on casper02


# Imports and Options

In [2]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

import os
# from mpl_toolkits.basemap import Basemap  #2020: commented out because appears to be a missing file
import matplotlib.pyplot as plt
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import xarray as xr

from loca.data_catalog import load_monthly_cmip_hydro_datasets, resample_monthly_data
from loca.utils import calc_change


#cache is written in script "write_cache.ipynb"  TODO
read_cache = False
# cache_dir = os.environ['TMPDIR']
# cache_dir = '/glade/u/home/jvano/scratch/'  #old

cache_dir = '/glade/scratch/jvano'  #new in 2020


# TODO still need to clean this up, possibly add others - or migrate this script to another place.  
# LIke that have ability to do individual models

# CACHED:
# models = ['ACCESS1-0', 'CanESM2']
# models = ['MIROC5']
# models = ['MRI-CGCM3']
# models = ['bcc-csm1-1']
# models = ['bcc-csm1-1-m']
# models = ['CCSM4']
# models = ['CESM1-BGC']
# models = ['CMCC-CM']
# models = ['GISS-E2-R']
# models = ['HadGEM2-CC']
# models = ['HadGEM2-ES', 'inmcm4']
# models = ['IPSL-CM5A-MR']
# models = ['MIROC-ESM', 'MIROC-ESM-CHEM']
# models = ['MPI-ESM-LR']
# models = ['MPI-ESM-MR']
# models = ['GFDL-ESM2G'] 
# models = ['NorESM1-M']
# models = ['MIROC5', 'MRI-CGCM3', 'bcc-csm1-1', 'bcc-csm1-1-m', 'CCSM4']
# models = ['CESM1-BGC', 'CESM1-CAM5', 'CMCC-CM', 'GISS-E2-R', 'HadGEM2-CC','HadGEM2-ES', 'inmcm4']
# models = ['IPSL-CM5A-MR']
# models = ['MIROC-ESM', 'MIROC-ESM-CHEM']
# models = ['MPI-ESM-LR', 'MPI-ESM-MR', 'NorESM1-M']
# models = ['GFDL-ESM2G']
# models = ['CNRM-CM5']
# models = ['CSIRO-Mk3-6-0']
# models = ['GFDL-ESM2M'] 
# models = ['GFDL-CM3'] 
# models = ['HadGEM2-AO']


# list of 25, without issues but CCSM4 and GISS-E2-R are not used in analysis:
# models = ['ACCESS1-0', 'CanESM2', 'MIROC5', 'MRI-CGCM3', 'bcc-csm1-1', 'bcc-csm1-1-m', 'CCSM4',
#           'CESM1-BGC', 'CMCC-CM', 'GISS-E2-R', 'HadGEM2-CC', 'HadGEM2-ES', 'inmcm4', 'IPSL-CM5A-MR',
#           'MIROC-ESM', 'MIROC-ESM-CHEM', 'MPI-ESM-LR', 'MPI-ESM-MR', 'NorESM1-M', 'GFDL-ESM2G', 
#           'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GFDL-ESM2M','GFDL-CM3', 'HadGEM2-AO']

# list of 23:
models = ['ACCESS1-0', 'CanESM2', 'MIROC5', 'MRI-CGCM3', 'bcc-csm1-1', 'bcc-csm1-1-m',
          'CESM1-BGC', 'CMCC-CM', 'HadGEM2-CC', 'HadGEM2-ES', 'inmcm4', 'IPSL-CM5A-MR',
          'MIROC-ESM', 'MIROC-ESM-CHEM', 'MPI-ESM-LR', 'MPI-ESM-MR', 'NorESM1-M', 'GFDL-ESM2G', 
          'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GFDL-ESM2M','GFDL-CM3', 'HadGEM2-AO']

# models = ['ACCESS1-0']
# models = ['MIROC5', 'MRI-CGCM3', 'bcc-csm1-1', 'bcc-csm1-1-m', 'CCSM4']
# models = ['CESM1-BGC', 'CMCC-CM', 'GISS-E2-R', 'HadGEM2-CC', 'inmcm4', 'IPSL-CM5A-MR']
# models = ['MIROC-ESM', 'MIROC-ESM-CHEM', 'MPI-ESM-LR', 'MPI-ESM-MR', 'NorESM1-M', 'GFDL-ESM2G'] 
models = ['CNRM-CM5', 'CSIRO-Mk3-6-0', 'GFDL-ESM2M','GFDL-CM3', 'HadGEM2-AO']

In [3]:
from dask.distributed import Client
client = Client(scheduler_file=os.path.join(os.environ['HOME'], 'scheduler_file.json'))
client

Client Scheduler: tcp://10.12.205.11:45598 Dashboard: proxy/8787/status,Cluster Workers: 2 Cores: 4 Memory: 60.00 GB


In [4]:
# OBSERVATIONS

#TODO CHANGE SO WRITE ALL, NOT JUST FUT HYDRO
read_cache = False

from loca.data_catalog import load_monthly_historical_hydro_datasets

hist_cmip_data = {}
if read_cache:
    print("reading values from cache")
    for key in ['loca', 'bcsd']:
        hist_cmip_data[key] = xr.open_mfdataset(
            os.path.join(cache_dir, f'monthly_cmip_hydro_hist.{key}_*.nc'),
            concat_dim='gcm', chunks={'time': 72})
    print("finished reading from cache")
    #TODO once get obs load separate won't have to do work around below
    single = ['ACCESS1-0']
    obs_data = load_monthly_historical_hydro_datasets(models=single, autoclose=True, parallel=True)
    #obs_data = load_monthly_historical_hydro_datasets(single, autoclose=True, parallel=True) #appears to work without "models"
    print("finished reading obs")
    # Merge two
    x = obs_data
    y = hist_cmip_data
    hydro_data = {**x, **y}

# if write_cache:
#     hydro_data['livneh'].sel().load().to_netcdf(os.path.join(cache_dir, f'monthly_hydro_obs.livneh.nc'))
#     hydro_data['maurer'].sel().load().to_netcdf(os.path.join(cache_dir, f'monthly_hydro_obs.maurer.nc'))

In [15]:
#TODO why below not working, ask Joe  (or don't really need to change this as long as can cache it)
# from loca.data_catalog import load_monthly_obs_hydro_datasets
# obs_data = load_monthly_obs_hydro_datasets(autoclose=True, parallel=True)

# Load Data

In [5]:
write_cache = True  
models

if write_cache:
    xr.set_options(file_cache_maxsize=12000)
    hist_data = load_monthly_cmip_hydro_datasets('historical', models=models, autoclose=False, parallel=True)
    rcp4_data = load_monthly_cmip_hydro_datasets('rcp45', models=models, autoclose=False, parallel=True)
    rcp8_data = load_monthly_cmip_hydro_datasets('rcp85', models=models, autoclose=False, parallel=True)

load_monthly_cmip_hydro_datasets
load_monthly_loca_hydrology
load_daily_loca_hydrology
load_monthly_bcsd_hydrology
load_bcsd_dataset
load_monthly_cmip_hydro_datasets
load_monthly_loca_hydrology
load_daily_loca_hydrology
load_monthly_bcsd_hydrology
load_bcsd_dataset
load_monthly_cmip_hydro_datasets
load_monthly_loca_hydrology
load_daily_loca_hydrology
load_monthly_bcsd_hydrology
load_bcsd_dataset


In [6]:
if write_cache:
    for key, ds in hist_data.items():
        for gcm in models:
            ds.sel(gcm=gcm).load().to_netcdf(os.path.join(os.environ['TMPDIR'], f'monthly_cmip_hydro_hist.{key}_{gcm}.nc'))
    for key, ds in rcp4_data.items():
        for gcm in models:
            ds.sel(gcm=gcm).load().to_netcdf(os.path.join(os.environ['TMPDIR'], f'monthly_cmip_hydro_rcp4.{key}_{gcm}.nc'))
    for key, ds in rcp8_data.items():
        for gcm in models:
            ds.sel(gcm=gcm).load().to_netcdf(os.path.join(os.environ['TMPDIR'], f'monthly_cmip_hydro_rcp8.{key}_{gcm}.nc'))
#     for key, ds in obs_data.items():
#             ds.sel()

NameError: name 'obs_data' is not defined

In [8]:
#test to see if reads in

read_cache = True
hist_data = {}
rcp4_data = {}
rcp8_data = {}
if read_cache:
    for key in ['loca', 'bcsd']:
        hist_data[key] = xr.open_mfdataset(
            os.path.join(cache_dir, f'monthly_cmip_hydro_hist.{key}_*.nc'),
            concat_dim='gcm', chunks={'time': 72})
        rcp4_data[key] = xr.open_mfdataset(
            os.path.join(cache_dir, f'monthly_cmip_hydro_rcp4.{key}_*.nc'),
            concat_dim='gcm', chunks={'time': 72})
        rcp8_data[key] = xr.open_mfdataset(
            os.path.join(cache_dir, f'monthly_cmip_hydro_rcp8.{key}_*.nc'),
            concat_dim='gcm', chunks={'time': 72})

In [11]:
rcp8_data

{'loca': <xarray.Dataset>
 Dimensions:       (gcm: 25, lat: 224, lon: 464, time: 1140)
 Coordinates:
   * lat           (lat) float64 25.06 25.19 25.31 25.44 ... 52.69 52.81 52.94
   * lon           (lon) float64 -124.9 -124.8 -124.7 ... -67.31 -67.19 -67.06
   * time          (time) datetime64[ns] 2006-01-01 2006-02-01 ... 2100-12-01
   * gcm           (gcm) <U14 'ACCESS1-0' 'CCSM4' ... 'bcc-csm1-1' 'inmcm4'
 Data variables:
     ET            (gcm, time, lat, lon) float32 dask.array<shape=(25, 1140, 224, 464), chunksize=(1, 72, 224, 464)>
     total_runoff  (gcm, time, lat, lon) float32 dask.array<shape=(25, 1140, 224, 464), chunksize=(1, 72, 224, 464)>
     SWE           (gcm, time, lat, lon) float32 dask.array<shape=(25, 1140, 224, 464), chunksize=(1, 72, 224, 464)>,
 'bcsd': <xarray.Dataset>
 Dimensions:       (gcm: 25, lat: 222, lon: 462, time: 1128)
 Coordinates:
   * lon           (lon) float32 -124.6875 -124.5625 ... -67.1875 -67.0625
   * lat           (lat) float32 25.1875 2